# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [25]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [26]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Barletta, M., Ponton, C. - La question de la normalisation des Ã©crits scolaires pour leur traitement automatique. Le cas de l'omission de mots.	Talk	talk-1	Bruit de fond ou valeur ajoutÃ©e? GÃ©rer le bruit lors des traitements informatiques des corpus linguistiques	2023-04-28	Grenoble, France	https://hal.science/hal-04090669	PrÃ©sentation donnÃ©e lors de la JournÃ©e dâ€™Ã©tudes â€œBruit de fond ou valeur ajoutÃ©?â€�
Barletta, M. - Outiller les chaÃ®nes de (co)rÃ©fÃ©rencedans un corpus dâ€™Ã©crits scolaires	Poster	poster-1	Ecole dâ€™Ã©tÃ© ETAL 2023	2023-06-16	Marseille, France	https://www.researchgate.net/publication/381310948_poster_ecole_d'ete_ETAL_2023_Marseille_juin_2023	Poster prÃ©sentÃ© lors de lâ€™Ã©cole dâ€™Ã©tÃ© ETAL 2023.
Barletta, M., Ponton, C. - Outiller l'Ã©tude des chaines de rÃ©fÃ©rence dans des Ã©crits scolaires 	Poster	poster-2	JournÃ©es de Linguistique des Corpus	2023-07-05	Grenoble, France	https://hal.scie

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [32]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0, encoding="utf8")
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,"Barletta, M., Ponton, C. - La question de la n...",Talk,talk-1,Bruit de fond ou valeur ajoutée? Gérer le brui...,2023-04-28,"Grenoble, France",https://hal.science/hal-04090669,Présentation donnée lors de la Journée d’étude...
1,"Barletta, M. - Outiller les chaînes de (co)réf...",Poster,poster-1,Ecole d’été ETAL 2023,2023-06-16,"Marseille, France",https://www.researchgate.net/publication/38131...,Poster présenté lors de l’école d’été ETAL 2023.
2,"Barletta, M., Ponton, C. - Outiller l'étude de...",Poster,poster-2,Journées de Linguistique des Corpus,2023-07-05,"Grenoble, France",https://hal.science/hal-04153289,Poster sur le processus d’annotation en contin...
3,"Barletta, M. - Annotation outillée de la conti...",Poster,poster-3,35èmes Journées d'Études sur la Parole (JEP) 3...,2024-07-10,"Toulouse, France",https://hal.science/hal-04646867,Poster présenté lors de la session poster TANL...
4,"Barletta, M. - Annotation de la continuité réf...",Talk,talk-2,35èmes Journées d'Études sur la Parole (JEP) 3...,2024-07-10,"Toulouse, France",https://inria.hal.science/hal-04622985,Présentation donnée lors de la session TALN/RE...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [28]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [29]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w', encoding="utf-8") as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [30]:
!ls ../_talks

2023-04-28-talk-1.md
2023-06-16-poster-1.md
2023-07-05-poster-2.md
2024-07-10-poster-3.md
2024-07-10-talk-2.md


In [31]:
!cat ../_talks/2023-04-28-talk-1.md

---
title: "Barletta, M., Ponton, C. - La question de la normalisation des Ã©crits scolaires pour leur traitement automatique. Le cas de l'omission de mots."
collection: talks
type: "Talk"
permalink: /talks/2023-04-28-talk-1
venue: "Bruit de fond ou valeur ajoutÃ©e? GÃ©rer le bruit lors des traitements informatiques des corpus linguistiques"
date: 2023-04-28
location: "Grenoble, France"
---

[More information here](https://hal.science/hal-04090669)

PrÃ©sentation donnÃ©e lors de la JournÃ©e dâ€™Ã©tudes â€œBruit de fond ou valeur ajoutÃ©?â€�
